# E-Ticaret Veri Analizi ve Makine Ogrenmesi Projesi

## Noel Donemi Perakende Satis ve Lojistik Analizi

---

**Veri Seti:** Christmas Retail Sales - Shipping & Delivery Dataset (Kaggle)

**Amac:** Bu calismada, e-ticaret lojistik verileri uzerinde veri biliminin temel adimlarini uygulayarak:
- Veri temizleme ve on isleme
- Kesifsel veri analizi (EDA)
- Istatistiksel analizler
- Gorsellestirme teknikleri
- Makine ogrenmesi modelleri

konularini kapsamli bir sekilde inceleyecegiz.

---

**Hazirlayan:** Ogrenci

**Ders:** Veri Bilimine Giris ve Makine Ogrenmesi

---

# BOLUM 1: KUTUPHANELERIN YUKLENMESI

Veri analizi, gorsellestirme ve makine ogrenmesi icin gerekli Python kutuphanelerini import ediyoruz.

In [ ]:
# =============================================================================
# TEMEL VERI ISLEME KUTUPHANELERI
# =============================================================================
import pandas as pd                    # Veri manipulasyonu icin
import numpy as np                     # Sayisal islemler icin

# =============================================================================
# GORSELLESTIRME KUTUPHANELERI
# =============================================================================
import matplotlib.pyplot as plt        # Temel grafik kutuphanesi
import seaborn as sns                  # Istatistiksel gorsellestirme

# =============================================================================
# MAKINE OGRENMESI KUTUPHANELERI
# =============================================================================
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, confusion_matrix, 
                             classification_report, roc_curve, auc)

# =============================================================================
# ISTATISTIK KUTUPHANELERI
# =============================================================================
from scipy import stats

# =============================================================================
# UYARILAR VE GRAFIK AYARLARI
# =============================================================================
import warnings
warnings.filterwarnings('ignore')      # Uyarilari gizle

# Grafik stilini ayarlayalim
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Seaborn renk paleti
sns.set_palette('husl')

print("Tum kutuphaneler basariyla yuklendi!")
print(f"Pandas surumu: {pd.__version__}")
print(f"NumPy surumu: {np.__version__}")

---

# BOLUM 2: VERI SETININ YUKLENMESI VE TANITIMI

## 2.1 Veri Setinin Genel Amaci

Bu veri seti, Black Friday, Cyber Monday ve Noel donemlerindeki perakende satis verilerini icermektedir. 

**Tablolar:**
| Tablo | Aciklama | Is Sureci |
|-------|----------|----------|
| OrderHeader | Siparis baslik bilgileri | Musteri siparisi olusturma |
| OrderLine | Siparis satir detaylari | Urun bazli siparis icerigi |
| Product | Urun bilgileri | Urun katalogu |
| Promotion | Kampanya bilgileri | Pazarlama kampanyalari |
| Fulfillment | Sevkiyat/teslimat bilgileri | Lojistik operasyonlari |
| Returns | Iade bilgileri | Iade yonetimi |
| Calendar | Tarih boyutu | Zaman analizi |

In [ ]:
# =============================================================================
# EXCEL DOSYASININ YUKLENMESI
# =============================================================================

# Dosya yolu
dosya_yolu = "Christmas_Retail_Sales_and_Marketing_Analysis_Dataset.xlsx"

# Tum tablolari yukleyelim
print("Veri seti yukleniyor...")
print("=" * 60)

# Excel dosyasindaki tum sheet'leri oku
excel_file = pd.ExcelFile(dosya_yolu)

# Her tabloyu ayri degiskene atayalim
order_line = pd.read_excel(excel_file, sheet_name='OrderLine')
product = pd.read_excel(excel_file, sheet_name='Product')
promotion = pd.read_excel(excel_file, sheet_name='Promotion')
returns = pd.read_excel(excel_file, sheet_name='Returns')
calendar = pd.read_excel(excel_file, sheet_name='Calendar')
fulfillment = pd.read_excel(excel_file, sheet_name='Fulfillment')
order_header = pd.read_excel(excel_file, sheet_name='OrderHeader')

print("Tum tablolar basariyla yuklendi!")

In [ ]:
# =============================================================================
# TABLOLARIN BOYUTLARI
# =============================================================================

# Tablo bilgilerini bir sozlukte toplayalim
tablolar = {
    'OrderHeader': order_header,
    'OrderLine': order_line,
    'Product': product,
    'Promotion': promotion,
    'Fulfillment': fulfillment,
    'Returns': returns,
    'Calendar': calendar
}

# Ozet tablo olusturalim
ozet_liste = []
for isim, tablo in tablolar.items():
    ozet_liste.append({
        'Tablo Adi': isim,
        'Satir Sayisi': f"{tablo.shape[0]:,}",
        'Sutun Sayisi': tablo.shape[1],
        'Bellek (KB)': f"{tablo.memory_usage(deep=True).sum() / 1024:.1f}"
    })

ozet_df = pd.DataFrame(ozet_liste)

print("VERI SETI OZET BILGILERI")
print("=" * 60)
display(ozet_df)

# Toplam kayit sayisi
toplam = sum([t.shape[0] for t in tablolar.values()])
print(f"\nToplam kayit sayisi: {toplam:,}")

## 2.2 Tablolarin Detayli Incelenmesi

In [ ]:
# =============================================================================
# FULFILLMENT TABLOSU (ANA ODAK NOKTAMIZ)
# =============================================================================
# Bu tablo teslimat bilgilerini icerir - makine ogrenmesi modelimizin hedefi burada

print("FULFILLMENT TABLOSU - Sevkiyat ve Teslimat Bilgileri")
print("=" * 60)
print(f"Boyut: {fulfillment.shape[0]:,} satir x {fulfillment.shape[1]} sutun\n")

display(fulfillment.head(10))

print("\nSutun Bilgileri:")
print("-" * 40)
for col in fulfillment.columns:
    print(f"  - {col}: {fulfillment[col].dtype}")

In [ ]:
# =============================================================================
# ORDER HEADER TABLOSU
# =============================================================================

print("ORDER HEADER TABLOSU - Siparis Baslik Bilgileri")
print("=" * 60)
display(order_header.head())

print("\nOdeme Tipleri Dagilimi:")
print(order_header['PaymentType'].value_counts())

In [ ]:
# =============================================================================
# PRODUCT TABLOSU
# =============================================================================

print("PRODUCT TABLOSU - Urun Bilgileri")
print("=" * 60)
display(product.head())

print("\nKategoriler:")
print(product['Category'].value_counts())

In [ ]:
# =============================================================================
# RETURNS TABLOSU
# =============================================================================

print("RETURNS TABLOSU - Iade Bilgileri")
print("=" * 60)
display(returns.head())

print("\nIade Nedenleri:")
print(returns['ReasonCode'].value_counts())

---

# BOLUM 3: VERI ON ISLEME

Bu bolumde verileri analiz ve modelleme icin hazirliyoruz.

## 3.1 Eksik Deger Analizi

In [ ]:
# =============================================================================
# TUM TABLOLAR ICIN EKSIK DEGER ANALIZI
# =============================================================================

print("EKSIK DEGER ANALIZI")
print("=" * 60)

for isim, tablo in tablolar.items():
    eksik = tablo.isnull().sum()
    eksik_var = eksik[eksik > 0]
    
    if len(eksik_var) > 0:
        print(f"\n{isim}:")
        for col, sayi in eksik_var.items():
            yuzde = (sayi / len(tablo)) * 100
            print(f"  - {col}: {sayi:,} eksik (%{yuzde:.2f})")
    else:
        print(f"\n{isim}: Eksik deger yok")

In [ ]:
# =============================================================================
# EKSIK DEGERLERIN GORSELLESTIRILMESI
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Fulfillment eksik degerler
eksik_ful = fulfillment.isnull().sum()
colors = ['#e74c3c' if x > 0 else '#2ecc71' for x in eksik_ful]
axes[0, 0].barh(eksik_ful.index, eksik_ful.values, color=colors)
axes[0, 0].set_title('Fulfillment - Eksik Degerler', fontweight='bold')
axes[0, 0].set_xlabel('Eksik Deger Sayisi')

# OrderLine eksik degerler
eksik_ol = order_line.isnull().sum()
colors = ['#e74c3c' if x > 0 else '#2ecc71' for x in eksik_ol]
axes[0, 1].barh(eksik_ol.index, eksik_ol.values, color=colors)
axes[0, 1].set_title('OrderLine - Eksik Degerler', fontweight='bold')
axes[0, 1].set_xlabel('Eksik Deger Sayisi')

# Product eksik degerler
eksik_pr = product.isnull().sum()
colors = ['#e74c3c' if x > 0 else '#2ecc71' for x in eksik_pr]
axes[1, 0].barh(eksik_pr.index, eksik_pr.values, color=colors)
axes[1, 0].set_title('Product - Eksik Degerler', fontweight='bold')
axes[1, 0].set_xlabel('Eksik Deger Sayisi')

# Returns eksik degerler
eksik_rt = returns.isnull().sum()
colors = ['#e74c3c' if x > 0 else '#2ecc71' for x in eksik_rt]
axes[1, 1].barh(eksik_rt.index, eksik_rt.values, color=colors)
axes[1, 1].set_title('Returns - Eksik Degerler', fontweight='bold')
axes[1, 1].set_xlabel('Eksik Deger Sayisi')

plt.tight_layout()
plt.suptitle('Tablolardaki Eksik Deger Durumu', fontsize=16, fontweight='bold', y=1.02)
plt.show()

print("Yesil: Eksik deger yok | Kirmizi: Eksik deger var")

## 3.2 Veri Tiplerinin Duzenlenmesi

In [ ]:
# =============================================================================
# TARIH SUTUNLARININ DATETIME'A DONUSTURULMESI
# =============================================================================

# Fulfillment tablosundaki tarihler
fulfillment['PromisedDate'] = pd.to_datetime(fulfillment['PromisedDate'])
fulfillment['ShipDate'] = pd.to_datetime(fulfillment['ShipDate'])
fulfillment['DeliveryDate'] = pd.to_datetime(fulfillment['DeliveryDate'])

# OrderHeader tablosundaki tarih
order_header['OrderDateTime'] = pd.to_datetime(order_header['OrderDateTime'])

# Returns tablosundaki tarih
returns['ReturnDate'] = pd.to_datetime(returns['ReturnDate'])

# Calendar tablosundaki tarih
calendar['Date'] = pd.to_datetime(calendar['Date'])

print("Tarih sutunlari datetime formatina donusturuldu!")
print("\nFulfillment veri tipleri:")
print(fulfillment.dtypes)

## 3.3 Tablolarin Birlestirilmesi (Merge)

In [ ]:
# =============================================================================
# ANA ANALIZ TABLOSUNUN OLUSTURULMASI
# =============================================================================
# Fulfillment + OrderHeader + OrderLine + Product tablolarini birlestiriyoruz

# 1. Fulfillment + OrderHeader
df = fulfillment.merge(order_header, on='OrderID', how='left')
print(f"Fulfillment + OrderHeader: {df.shape}")

# 2. OrderLine'dan siparis bazli ozet bilgiler alalim
order_ozet = order_line.groupby('OrderID').agg({
    'Qty': 'sum',
    'LineRevenue$': 'sum',
    'LineCost$': 'sum',
    'LineDiscount$': 'sum',
    'OrderLineID': 'count'
}).reset_index()

order_ozet.columns = ['OrderID', 'ToplamAdet', 'ToplamGelir', 'ToplamMaliyet', 'ToplamIndirim', 'UrunCesidi']

# 3. Ana tabloya ekleyelim
df = df.merge(order_ozet, on='OrderID', how='left')
print(f"+ Order Ozet: {df.shape}")

print(f"\nBirlesitirilmis veri seti boyutu: {df.shape[0]:,} satir x {df.shape[1]} sutun")
display(df.head())

---

# BOLUM 4: OZELLIK MUHENDISLIGI (FEATURE ENGINEERING)

Makine ogrenmesi modelimiz icin yeni ve anlamli ozellikler turetiyoruz.

In [ ]:
# =============================================================================
# SURE BAZLI OZELLIKLER
# =============================================================================

# 1. Tasima Suresi (gun): Teslimat Tarihi - Sevkiyat Tarihi
df['TasimaSuresi'] = (df['DeliveryDate'] - df['ShipDate']).dt.days

# 2. Taahhut Edilen Sure (gun): Vaat Edilen Tarih - Sevkiyat Tarihi
df['TaahhutSuresi'] = (df['PromisedDate'] - df['ShipDate']).dt.days

# 3. Siparis-Sevkiyat Suresi (gun): Sevkiyat Tarihi - Siparis Tarihi
df['SiparisSevkSuresi'] = (df['ShipDate'] - df['OrderDateTime']).dt.days

# 4. Gecikme Gun Sayisi
df['GecikmeGunu'] = (df['DeliveryDate'] - df['PromisedDate']).dt.days
df['GecikmeGunu'] = df['GecikmeGunu'].apply(lambda x: max(0, x) if pd.notna(x) else 0)

# 5. Erken Teslimat Gun Sayisi
df['ErkenTeslimat'] = (df['PromisedDate'] - df['DeliveryDate']).dt.days
df['ErkenTeslimat'] = df['ErkenTeslimat'].apply(lambda x: max(0, x) if pd.notna(x) else 0)

print("Sure bazli ozellikler olusturuldu:")
print("  - TasimaSuresi")
print("  - TaahhutSuresi")
print("  - SiparisSevkSuresi")
print("  - GecikmeGunu")
print("  - ErkenTeslimat")

In [ ]:
# =============================================================================
# TARIH BAZLI OZELLIKLER
# =============================================================================

# Siparis tarihinden ozellik cikarimi
df['SiparisAyi'] = df['OrderDateTime'].dt.month
df['SiparisGunu'] = df['OrderDateTime'].dt.dayofweek  # 0=Pazartesi
df['SiparisHaftasi'] = df['OrderDateTime'].dt.isocalendar().week
df['SiparisSaati'] = df['OrderDateTime'].dt.hour

# Haftasonu kontrolu
df['HaftaSonu'] = df['SiparisGunu'].apply(lambda x: 1 if x >= 5 else 0)

# Ay isimlerini ekleyelim (gorsellestirme icin)
ay_isimleri = {1:'Ocak', 2:'Subat', 3:'Mart', 4:'Nisan', 5:'Mayis', 6:'Haziran',
               7:'Temmuz', 8:'Agustos', 9:'Eylul', 10:'Ekim', 11:'Kasim', 12:'Aralik'}
df['AyAdi'] = df['SiparisAyi'].map(ay_isimleri)

# Gun isimlerini ekleyelim
gun_isimleri = {0:'Pazartesi', 1:'Sali', 2:'Carsamba', 3:'Persembe', 
                4:'Cuma', 5:'Cumartesi', 6:'Pazar'}
df['GunAdi'] = df['SiparisGunu'].map(gun_isimleri)

print("Tarih bazli ozellikler olusturuldu:")
print("  - SiparisAyi, AyAdi")
print("  - SiparisGunu, GunAdi")
print("  - SiparisHaftasi")
print("  - SiparisSaati")
print("  - HaftaSonu")

In [ ]:
# =============================================================================
# FINANSAL OZELLIKLER
# =============================================================================

# Kar marji
df['KarMarji'] = df['ToplamGelir'] - df['ToplamMaliyet']

# Kar yuzdesi
df['KarYuzdesi'] = (df['KarMarji'] / df['ToplamGelir'] * 100).round(2)

# Indirim orani
df['IndirimOrani'] = (df['ToplamIndirim'] / (df['ToplamGelir'] + df['ToplamIndirim']) * 100).round(2)

# Ortalama urun fiyati
df['OrtalamaUrunFiyati'] = (df['ToplamGelir'] / df['ToplamAdet']).round(2)

# Kargo maliyeti / Gelir orani
df['KargoGelirOrani'] = (df['ShipCost$'] / df['ToplamGelir'] * 100).round(2)

print("Finansal ozellikler olusturuldu:")
print("  - KarMarji")
print("  - KarYuzdesi")
print("  - IndirimOrani")
print("  - OrtalamaUrunFiyati")
print("  - KargoGelirOrani")

In [ ]:
# =============================================================================
# OLUSTURULAN OZELLIKLERIN OZETI
# =============================================================================

yeni_ozellikler = ['TasimaSuresi', 'TaahhutSuresi', 'SiparisSevkSuresi', 'GecikmeGunu', 
                   'ErkenTeslimat', 'KarMarji', 'KarYuzdesi', 'IndirimOrani']

print("YENI OZELLIKLERIN ISTATISTIKLERI")
print("=" * 60)
display(df[yeni_ozellikler].describe().round(2))

---

# BOLUM 5: KESIFSEL VERI ANALIZI (EDA)

Verilerimizi detayli olarak inceliyor ve gorsellestiriyoruz.

## 5.1 Hedef Degisken Analizi: Teslimat Durumu

In [ ]:
# =============================================================================
# TESLIMAT DURUMU DAGILIMI
# =============================================================================

print("TESLIMAT DURUMU DAGILIMI")
print("=" * 40)
teslimat_dagilim = df['DeliveryStatus'].value_counts()
print(teslimat_dagilim)
print(f"\nToplam: {teslimat_dagilim.sum():,}")

# Yuzdelik dagilim
print("\nYuzdelik Dagilim:")
print((df['DeliveryStatus'].value_counts(normalize=True) * 100).round(2))

In [ ]:
# =============================================================================
# TESLIMAT DURUMU GORSELLESTIRMESI
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Pasta Grafigi
colors = ['#2ecc71', '#e74c3c']
explode = (0.02, 0.02)
axes[0].pie(teslimat_dagilim.values, labels=teslimat_dagilim.index, autopct='%1.1f%%',
            colors=colors, explode=explode, shadow=True, startangle=90)
axes[0].set_title('Teslimat Durumu Dagilimi\n(Pasta Grafigi)', fontweight='bold')

# 2. Bar Grafigi
bars = axes[1].bar(teslimat_dagilim.index, teslimat_dagilim.values, color=colors, edgecolor='black')
axes[1].set_xlabel('Teslimat Durumu')
axes[1].set_ylabel('Siparis Sayisi')
axes[1].set_title('Teslimat Durumu Dagilimi\n(Bar Grafigi)', fontweight='bold')
for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                 f'{int(height):,}', ha='center', va='bottom', fontweight='bold')

# 3. Donut Grafigi
wedges, texts, autotexts = axes[2].pie(teslimat_dagilim.values, labels=teslimat_dagilim.index,
                                        autopct='%1.1f%%', colors=colors, 
                                        wedgeprops=dict(width=0.5), startangle=90)
axes[2].set_title('Teslimat Durumu Dagilimi\n(Donut Grafigi)', fontweight='bold')

plt.tight_layout()
plt.show()

## 5.2 Kargo Firmasi Analizi

In [ ]:
# =============================================================================
# KARGO FIRMALARINA GORE ANALIZ
# =============================================================================

print("KARGO FIRMALARI DAGILIMI")
print("=" * 40)
print(df['Carrier'].value_counts())

In [ ]:
# =============================================================================
# KARGO FIRMASI PERFORMANS ANALIZI
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Kargo firmasi dagilimi
kargo_dagilim = df['Carrier'].value_counts()
colors = sns.color_palette('husl', len(kargo_dagilim))
axes[0, 0].bar(kargo_dagilim.index, kargo_dagilim.values, color=colors, edgecolor='black')
axes[0, 0].set_xlabel('Kargo Firmasi')
axes[0, 0].set_ylabel('Siparis Sayisi')
axes[0, 0].set_title('Kargo Firmasi Dagilimi', fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Kargo firmasina gore gecikme orani
gecikme_orani = df.groupby('Carrier')['DeliveryStatus'].apply(
    lambda x: (x == 'Late').sum() / len(x) * 100
).sort_values(ascending=False)

colors = ['#e74c3c' if x > 50 else '#f39c12' if x > 30 else '#2ecc71' for x in gecikme_orani.values]
axes[0, 1].barh(gecikme_orani.index, gecikme_orani.values, color=colors, edgecolor='black')
axes[0, 1].set_xlabel('Gecikme Orani (%)')
axes[0, 1].set_title('Kargo Firmasina Gore Gecikme Orani', fontweight='bold')
axes[0, 1].axvline(x=50, color='red', linestyle='--', alpha=0.7, label='%50 Esik')
for i, v in enumerate(gecikme_orani.values):
    axes[0, 1].text(v + 1, i, f'{v:.1f}%', va='center', fontweight='bold')

# 3. Kargo firmasina gore ortalama tasima suresi
tasima_suresi = df.groupby('Carrier')['TasimaSuresi'].mean().sort_values(ascending=False)
colors = sns.color_palette('YlOrRd', len(tasima_suresi))
axes[1, 0].barh(tasima_suresi.index, tasima_suresi.values, color=colors, edgecolor='black')
axes[1, 0].set_xlabel('Ortalama Tasima Suresi (Gun)')
axes[1, 0].set_title('Kargo Firmasina Gore Ort. Tasima Suresi', fontweight='bold')
for i, v in enumerate(tasima_suresi.values):
    axes[1, 0].text(v + 0.1, i, f'{v:.1f}', va='center')

# 4. Kargo firmasina gore ortalama kargo maliyeti
kargo_maliyet = df.groupby('Carrier')['ShipCost$'].mean().sort_values(ascending=False)
colors = sns.color_palette('Blues_r', len(kargo_maliyet))
axes[1, 1].barh(kargo_maliyet.index, kargo_maliyet.values, color=colors, edgecolor='black')
axes[1, 1].set_xlabel('Ortalama Kargo Maliyeti ($)')
axes[1, 1].set_title('Kargo Firmasina Gore Ort. Maliyet', fontweight='bold')
for i, v in enumerate(kargo_maliyet.values):
    axes[1, 1].text(v + 0.2, i, f'${v:.2f}', va='center')

plt.tight_layout()
plt.show()

## 5.3 Hizmet Seviyesi Analizi

In [ ]:
# =============================================================================
# HIZMET SEVIYESI ANALIZI
# =============================================================================

print("HIZMET SEVIYELERI")
print("=" * 40)
print(df['ServiceLevel'].value_counts())

In [ ]:
# =============================================================================
# HIZMET SEVIYESI DETAYLI ANALIZ
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Hizmet seviyesi dagilimi (pasta)
hizmet_dagilim = df['ServiceLevel'].value_counts()
colors = sns.color_palette('Set2', len(hizmet_dagilim))
axes[0].pie(hizmet_dagilim.values, labels=hizmet_dagilim.index, autopct='%1.1f%%',
            colors=colors, startangle=90)
axes[0].set_title('Hizmet Seviyesi Dagilimi', fontweight='bold')

# 2. Hizmet seviyesine gore gecikme orani
hizmet_gecikme = df.groupby('ServiceLevel')['DeliveryStatus'].apply(
    lambda x: (x == 'Late').sum() / len(x) * 100
).sort_values(ascending=True)

colors = ['#2ecc71' if x < 30 else '#f39c12' if x < 50 else '#e74c3c' for x in hizmet_gecikme.values]
bars = axes[1].barh(hizmet_gecikme.index, hizmet_gecikme.values, color=colors, edgecolor='black')
axes[1].set_xlabel('Gecikme Orani (%)')
axes[1].set_title('Hizmet Seviyesine Gore Gecikme Orani', fontweight='bold')
for i, v in enumerate(hizmet_gecikme.values):
    axes[1].text(v + 1, i, f'{v:.1f}%', va='center', fontweight='bold')

# 3. Hizmet seviyesi ve teslimat durumu (stacked bar)
cross_tab = pd.crosstab(df['ServiceLevel'], df['DeliveryStatus'], normalize='index') * 100
cross_tab.plot(kind='bar', stacked=True, ax=axes[2], color=['#e74c3c', '#2ecc71'], edgecolor='black')
axes[2].set_xlabel('Hizmet Seviyesi')
axes[2].set_ylabel('Yuzde (%)')
axes[2].set_title('Hizmet Seviyesi vs Teslimat Durumu', fontweight='bold')
axes[2].legend(title='Durum', loc='upper right')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5.4 Zaman Serisi Analizi

In [ ]:
# =============================================================================
# ZAMANA GORE SIPARIS VE GECIKME ANALIZI
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Aylara gore siparis sayisi
aylik_siparis = df.groupby('SiparisAyi').size()
axes[0, 0].plot(aylik_siparis.index, aylik_siparis.values, marker='o', linewidth=2, 
                markersize=10, color='#3498db')
axes[0, 0].fill_between(aylik_siparis.index, aylik_siparis.values, alpha=0.3, color='#3498db')
axes[0, 0].set_xlabel('Ay')
axes[0, 0].set_ylabel('Siparis Sayisi')
axes[0, 0].set_title('Aylara Gore Siparis Sayisi', fontweight='bold')
axes[0, 0].set_xticks(range(1, 13))
axes[0, 0].set_xticklabels(['Oca','Sub','Mar','Nis','May','Haz','Tem','Agu','Eyl','Eki','Kas','Ara'])
axes[0, 0].grid(True, alpha=0.3)

# 2. Aylara gore gecikme orani
aylik_gecikme = df.groupby('SiparisAyi')['DeliveryStatus'].apply(
    lambda x: (x == 'Late').sum() / len(x) * 100
)
colors = ['#e74c3c' if x > 50 else '#f39c12' if x > 30 else '#2ecc71' for x in aylik_gecikme.values]
axes[0, 1].bar(aylik_gecikme.index, aylik_gecikme.values, color=colors, edgecolor='black')
axes[0, 1].axhline(y=aylik_gecikme.mean(), color='red', linestyle='--', label=f'Ortalama: {aylik_gecikme.mean():.1f}%')
axes[0, 1].set_xlabel('Ay')
axes[0, 1].set_ylabel('Gecikme Orani (%)')
axes[0, 1].set_title('Aylara Gore Gecikme Orani', fontweight='bold')
axes[0, 1].set_xticks(range(1, 13))
axes[0, 1].set_xticklabels(['Oca','Sub','Mar','Nis','May','Haz','Tem','Agu','Eyl','Eki','Kas','Ara'])
axes[0, 1].legend()

# 3. Haftanin gunlerine gore siparis
gun_sirasi = ['Pazartesi', 'Sali', 'Carsamba', 'Persembe', 'Cuma', 'Cumartesi', 'Pazar']
gunluk_siparis = df.groupby('GunAdi').size().reindex(gun_sirasi)
colors = ['#3498db']*5 + ['#e74c3c']*2
axes[1, 0].bar(gun_sirasi, gunluk_siparis.values, color=colors, edgecolor='black')
axes[1, 0].set_xlabel('Gun')
axes[1, 0].set_ylabel('Siparis Sayisi')
axes[1, 0].set_title('Haftanin Gunlerine Gore Siparis Sayisi', fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Saatlere gore siparis dagilimi
saatlik_siparis = df.groupby('SiparisSaati').size()
axes[1, 1].bar(saatlik_siparis.index, saatlik_siparis.values, color='#9b59b6', edgecolor='black')
axes[1, 1].set_xlabel('Saat')
axes[1, 1].set_ylabel('Siparis Sayisi')
axes[1, 1].set_title('Saatlere Gore Siparis Dagilimi', fontweight='bold')
axes[1, 1].set_xticks(range(0, 24))

plt.tight_layout()
plt.show()

## 5.5 Finansal Analizler

In [ ]:
# =============================================================================
# GELIR VE KAR ANALIZI
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Gelir dagilimi (histogram)
axes[0, 0].hist(df['ToplamGelir'].dropna(), bins=50, color='#2ecc71', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['ToplamGelir'].mean(), color='red', linestyle='--', 
                   label=f'Ortalama: ${df["ToplamGelir"].mean():.2f}')
axes[0, 0].axvline(df['ToplamGelir'].median(), color='blue', linestyle='--', 
                   label=f'Medyan: ${df["ToplamGelir"].median():.2f}')
axes[0, 0].set_xlabel('Toplam Gelir ($)')
axes[0, 0].set_ylabel('Frekans')
axes[0, 0].set_title('Siparis Gelir Dagilimi', fontweight='bold')
axes[0, 0].legend()

# 2. Kar marji dagilimi
axes[0, 1].hist(df['KarMarji'].dropna(), bins=50, color='#3498db', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(df['KarMarji'].mean(), color='red', linestyle='--',
                   label=f'Ortalama: ${df["KarMarji"].mean():.2f}')
axes[0, 1].set_xlabel('Kar Marji ($)')
axes[0, 1].set_ylabel('Frekans')
axes[0, 1].set_title('Kar Marji Dagilimi', fontweight='bold')
axes[0, 1].legend()

# 3. Odeme tiplerine gore ortalama siparis tutari
odeme_gelir = df.groupby('PaymentType')['ToplamGelir'].mean().sort_values(ascending=True)
colors = sns.color_palette('viridis', len(odeme_gelir))
axes[1, 0].barh(odeme_gelir.index, odeme_gelir.values, color=colors, edgecolor='black')
axes[1, 0].set_xlabel('Ortalama Siparis Tutari ($)')
axes[1, 0].set_title('Odeme Tipine Gore Ort. Siparis Tutari', fontweight='bold')
for i, v in enumerate(odeme_gelir.values):
    axes[1, 0].text(v + 5, i, f'${v:.2f}', va='center')

# 4. Aylara gore toplam gelir
aylik_gelir = df.groupby('SiparisAyi')['ToplamGelir'].sum() / 1000000  # Milyon $
axes[1, 1].bar(aylik_gelir.index, aylik_gelir.values, color='#e74c3c', edgecolor='black')
axes[1, 1].set_xlabel('Ay')
axes[1, 1].set_ylabel('Toplam Gelir (Milyon $)')
axes[1, 1].set_title('Aylara Gore Toplam Gelir', fontweight='bold')
axes[1, 1].set_xticks(range(1, 13))
axes[1, 1].set_xticklabels(['Oca','Sub','Mar','Nis','May','Haz','Tem','Agu','Eyl','Eki','Kas','Ara'])

plt.tight_layout()
plt.show()

# Ozet istatistikler
print("\nFINANSAL OZET")
print("=" * 50)
print(f"Toplam Gelir: ${df['ToplamGelir'].sum():,.2f}")
print(f"Toplam Kar: ${df['KarMarji'].sum():,.2f}")
print(f"Ortalama Siparis Tutari: ${df['ToplamGelir'].mean():,.2f}")
print(f"Ortalama Kar Marji: ${df['KarMarji'].mean():,.2f}")

## 5.6 Tasima Suresi Analizi

In [ ]:
# =============================================================================
# TASIMA SURESI DETAYLI ANALIZ
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Tasima suresi dagilimi (histogram)
axes[0, 0].hist(df['TasimaSuresi'].dropna(), bins=30, color='#3498db', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['TasimaSuresi'].mean(), color='red', linestyle='--',
                   label=f'Ortalama: {df["TasimaSuresi"].mean():.1f} gun')
axes[0, 0].set_xlabel('Tasima Suresi (Gun)')
axes[0, 0].set_ylabel('Frekans')
axes[0, 0].set_title('Tasima Suresi Dagilimi', fontweight='bold')
axes[0, 0].legend()

# 2. Teslimat durumuna gore boxplot
df.boxplot(column='TasimaSuresi', by='DeliveryStatus', ax=axes[0, 1])
axes[0, 1].set_xlabel('Teslimat Durumu')
axes[0, 1].set_ylabel('Tasima Suresi (Gun)')
axes[0, 1].set_title('Teslimat Durumuna Gore Tasima Suresi', fontweight='bold')
plt.suptitle('')

# 3. Kargo firmasina gore boxplot
df.boxplot(column='TasimaSuresi', by='Carrier', ax=axes[1, 0])
axes[1, 0].set_xlabel('Kargo Firmasi')
axes[1, 0].set_ylabel('Tasima Suresi (Gun)')
axes[1, 0].set_title('Kargo Firmasina Gore Tasima Suresi', fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
plt.suptitle('')

# 4. Hizmet seviyesine gore violin plot
sns.violinplot(data=df, x='ServiceLevel', y='TasimaSuresi', ax=axes[1, 1], palette='Set2')
axes[1, 1].set_xlabel('Hizmet Seviyesi')
axes[1, 1].set_ylabel('Tasima Suresi (Gun)')
axes[1, 1].set_title('Hizmet Seviyesine Gore Tasima Suresi (Violin Plot)', fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5.7 Korelasyon Analizi

In [ ]:
# =============================================================================
# KORELASYON MATRISI
# =============================================================================

# Sayisal sutunlari secelim
sayisal_sutunlar = ['TasimaSuresi', 'TaahhutSuresi', 'SiparisSevkSuresi', 'GecikmeGunu',
                    'ToplamAdet', 'ToplamGelir', 'ToplamMaliyet', 'KarMarji',
                    'ShipCost$', 'SiparisAyi', 'SiparisSaati', 'HaftaSonu']

# Sadece mevcut sutunlari alalim
mevcut_sutunlar = [col for col in sayisal_sutunlar if col in df.columns]

# Korelasyon matrisi
korelasyon = df[mevcut_sutunlar].corr()

# Gorsellestirme
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(korelasyon, dtype=bool))  # Ust ucgeni maskele
sns.heatmap(korelasyon, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
            fmt='.2f', square=True, linewidths=0.5, cbar_kws={'shrink': 0.8})
plt.title('Sayisal Degiskenler Arasi Korelasyon Matrisi', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# En yuksek korelasyonlar
print("\nEN YUKSEK KORELASYONLAR (mutlak deger)")
print("=" * 50)
kor_pairs = []
for i in range(len(korelasyon.columns)):
    for j in range(i+1, len(korelasyon.columns)):
        kor_pairs.append((korelasyon.columns[i], korelasyon.columns[j], 
                          korelasyon.iloc[i, j]))

kor_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
for pair in kor_pairs[:10]:
    print(f"{pair[0]} <-> {pair[1]}: {pair[2]:.3f}")

## 5.8 Iade Analizi

In [ ]:
# =============================================================================
# IADE VERISI ANALIZI
# =============================================================================

print("IADE ISTATISTIKLERI")
print("=" * 50)
print(f"Toplam iade sayisi: {len(returns):,}")
print(f"Toplam siparis satiri: {len(order_line):,}")
print(f"Iade orani: %{(len(returns)/len(order_line)*100):.2f}")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Iade nedenleri
iade_nedenleri = returns['ReasonCode'].value_counts()
colors = sns.color_palette('Set3', len(iade_nedenleri))
axes[0, 0].pie(iade_nedenleri.values, labels=iade_nedenleri.index, autopct='%1.1f%%',
               colors=colors, startangle=90)
axes[0, 0].set_title('Iade Nedenleri Dagilimi', fontweight='bold')

# 2. Urun durumu
urun_durumu = returns['Condition'].value_counts()
axes[0, 1].bar(urun_durumu.index, urun_durumu.values, color=sns.color_palette('husl', len(urun_durumu)), 
               edgecolor='black')
axes[0, 1].set_xlabel('Urun Durumu')
axes[0, 1].set_ylabel('Iade Sayisi')
axes[0, 1].set_title('Iade Edilen Urun Durumu', fontweight='bold')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Iade sonucu (disposition)
disposition = returns['Disposition'].value_counts()
axes[1, 0].barh(disposition.index, disposition.values, color=sns.color_palette('coolwarm', len(disposition)),
                edgecolor='black')
axes[1, 0].set_xlabel('Sayi')
axes[1, 0].set_title('Iade Sonrasi Islem', fontweight='bold')

# 4. Iade tutarlari dagilimi
axes[1, 1].hist(returns['Refund$'].dropna(), bins=30, color='#e74c3c', edgecolor='black', alpha=0.7)
axes[1, 1].axvline(returns['Refund$'].mean(), color='blue', linestyle='--',
                   label=f'Ortalama: ${returns["Refund$"].mean():.2f}')
axes[1, 1].set_xlabel('Iade Tutari ($)')
axes[1, 1].set_ylabel('Frekans')
axes[1, 1].set_title('Iade Tutari Dagilimi', fontweight='bold')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print(f"\nToplam Iade Tutari: ${returns['Refund$'].sum():,.2f}")
print(f"Ortalama Iade Tutari: ${returns['Refund$'].mean():.2f}")

## 5.9 Urun Kategorisi Analizi

In [ ]:
# =============================================================================
# URUN KATEGORISI ANALIZI
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Kategori dagilimi
kategori_dagilim = product['Category'].value_counts()
colors = sns.color_palette('Set2', len(kategori_dagilim))
axes[0, 0].pie(kategori_dagilim.values, labels=kategori_dagilim.index, autopct='%1.1f%%',
               colors=colors, startangle=90)
axes[0, 0].set_title('Urun Kategorisi Dagilimi', fontweight='bold')

# 2. Kategorilere gore ortalama fiyat
kategori_fiyat = product.groupby('Category')['MSRP'].mean().sort_values(ascending=True)
axes[0, 1].barh(kategori_fiyat.index, kategori_fiyat.values, 
                color=sns.color_palette('viridis', len(kategori_fiyat)), edgecolor='black')
axes[0, 1].set_xlabel('Ortalama Fiyat ($)')
axes[0, 1].set_title('Kategorilere Gore Ortalama Fiyat', fontweight='bold')
for i, v in enumerate(kategori_fiyat.values):
    axes[0, 1].text(v + 1, i, f'${v:.2f}', va='center')

# 3. Mevsimsellik etiketi
sezon_dagilim = product['SeasonalityTag'].value_counts()
axes[1, 0].bar(sezon_dagilim.index, sezon_dagilim.values, 
               color=sns.color_palette('autumn', len(sezon_dagilim)), edgecolor='black')
axes[1, 0].set_xlabel('Mevsimsellik Etiketi')
axes[1, 0].set_ylabel('Urun Sayisi')
axes[1, 0].set_title('Mevsimsellik Etiketine Gore Urun Sayisi', fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Kar marji analizi (MSRP - StandardCost)
product['UrunKarMarji'] = product['MSRP'] - product['StandardCost']
axes[1, 1].hist(product['UrunKarMarji'].dropna(), bins=40, color='#2ecc71', edgecolor='black', alpha=0.7)
axes[1, 1].axvline(product['UrunKarMarji'].mean(), color='red', linestyle='--',
                   label=f'Ortalama: ${product["UrunKarMarji"].mean():.2f}')
axes[1, 1].set_xlabel('Urun Kar Marji ($)')
axes[1, 1].set_ylabel('Frekans')
axes[1, 1].set_title('Urun Kar Marji Dagilimi', fontweight='bold')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

---

# BOLUM 6: ISTATISTIKSEL TESTLER

Veri setimiz uzerinde istatistiksel testler yaparak anlamli cikarimlar elde ediyoruz.

In [ ]:
# =============================================================================
# NORMALLIK TESTI
# =============================================================================

print("NORMALLIK TESTI (Shapiro-Wilk)")
print("=" * 50)
print("Not: Buyuk veri setlerinde ornekleme yapildi (n=5000)\n")

# Ornekleme yapalim (Shapiro-Wilk buyuk veri setlerinde yavas calisir)
orneklem = df['TasimaSuresi'].dropna().sample(min(5000, len(df)), random_state=42)

stat, p_value = stats.shapiro(orneklem)
print(f"Tasima Suresi:")
print(f"  Test Istatistigi: {stat:.4f}")
print(f"  P-degeri: {p_value:.4e}")
print(f"  Sonuc: {'Normal dagilim DEGIL' if p_value < 0.05 else 'Normal dagilim'}")

In [ ]:
# =============================================================================
# T-TESTI: Hafta ici vs Hafta sonu tasima sureleri
# =============================================================================

print("\nBAGIMSIZ ORNEKLEM T-TESTI")
print("=" * 50)
print("H0: Hafta ici ve hafta sonu tasima sureleri arasinda fark yoktur")
print("H1: Hafta ici ve hafta sonu tasima sureleri arasinda fark vardir\n")

hafta_ici = df[df['HaftaSonu'] == 0]['TasimaSuresi'].dropna()
hafta_sonu = df[df['HaftaSonu'] == 1]['TasimaSuresi'].dropna()

t_stat, p_value = stats.ttest_ind(hafta_ici, hafta_sonu)

print(f"Hafta Ici Ortalama: {hafta_ici.mean():.2f} gun")
print(f"Hafta Sonu Ortalama: {hafta_sonu.mean():.2f} gun")
print(f"\nT-Istatistigi: {t_stat:.4f}")
print(f"P-degeri: {p_value:.4e}")
print(f"\nSonuc: {'Istatistiksel olarak ANLAMLI fark var' if p_value < 0.05 else 'Anlamli fark yok'}")

In [ ]:
# =============================================================================
# CHI-SQUARE TESTI: Kargo firmasi ve teslimat durumu bagimliligi
# =============================================================================

print("\nCHI-SQUARE BAGIMSIZLIK TESTI")
print("=" * 50)
print("H0: Kargo firmasi ve teslimat durumu birbirinden bagimsizdir")
print("H1: Kargo firmasi ve teslimat durumu arasinda iliski vardir\n")

# Capraz tablo
contingency_table = pd.crosstab(df['Carrier'], df['DeliveryStatus'])
print("Capraz Tablo:")
display(contingency_table)

chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"\nChi-Square Istatistigi: {chi2:.4f}")
print(f"Serbestlik Derecesi: {dof}")
print(f"P-degeri: {p_value:.4e}")
print(f"\nSonuc: {'Kargo firmasi ve teslimat durumu ILISKILI' if p_value < 0.05 else 'Bagimsiz'}")

---

# BOLUM 7: MAKINE OGRENMESI

## 7.1 Problem Tanimi

**Problem Turu:** Ikili Siniflandirma (Binary Classification)

**Hedef Degisken:** DeliveryStatus (Late / OnTime)

**Amac:** Bir siparisin teslimatinin gec olup olmayacagini tahmin etmek.

## 7.2 Veri Hazirligi

In [ ]:
# =============================================================================
# HEDEF DEGISKENIN HAZIRLANMASI
# =============================================================================

print("HEDEF DEGISKEN: DeliveryStatus")
print("=" * 50)
print(df['DeliveryStatus'].value_counts())

# Hedef degiskeni sayisallastiralim (Late=1, OnTime=0)
df['Hedef'] = (df['DeliveryStatus'] == 'Late').astype(int)

print("\nSayisallastirilmis Hedef:")
print(df['Hedef'].value_counts())
print(f"\nGecikme Orani: %{df['Hedef'].mean()*100:.2f}")

In [ ]:
# =============================================================================
# OZELLIK SECIMI VE KATEGORIK DEGISKENLERIN KODLANMASI
# =============================================================================

# Model icin kullanilacak ozellikler
ozellikler = ['Carrier', 'ServiceLevel', 'PaymentType', 'ChannelID',
              'ToplamAdet', 'ToplamGelir', 'ShipCost$', 
              'SiparisAyi', 'SiparisGunu', 'SiparisSaati', 'HaftaSonu']

# Mevcut ozellikleri kontrol edelim
mevcut_ozellikler = [col for col in ozellikler if col in df.columns]
print(f"Kullanilacak ozellik sayisi: {len(mevcut_ozellikler)}")

# Model veri setini olusturalim
df_model = df[mevcut_ozellikler + ['Hedef']].copy()

# Eksik degerleri temizleyelim
df_model = df_model.dropna()
print(f"Temizlenmis veri seti boyutu: {df_model.shape}")

In [ ]:
# =============================================================================
# KATEGORIK DEGISKENLERIN LABEL ENCODING ILE KODLANMASI
# =============================================================================

# Kategorik sutunlari tespit edelim
kategorik_sutunlar = df_model.select_dtypes(include=['object']).columns.tolist()
print(f"Kategorik sutunlar: {kategorik_sutunlar}")

# Label Encoder uygulayalim
label_encoders = {}
for col in kategorik_sutunlar:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col].astype(str))
    label_encoders[col] = le
    print(f"  {col}: {len(le.classes_)} sinif")

print("\nKodlama tamamlandi!")

In [ ]:
# =============================================================================
# X VE Y DEGISKENLERININ AYRILMASI
# =============================================================================

# Ozellikler (X) ve Hedef (y)
X = df_model.drop('Hedef', axis=1)
y = df_model['Hedef']

print(f"X (Ozellikler) boyutu: {X.shape}")
print(f"y (Hedef) boyutu: {y.shape}")
print(f"\nOzellikler: {list(X.columns)}")

## 7.3 Egitim ve Test Setlerinin Olusturulmasi

In [ ]:
# =============================================================================
# VERI SETININ BOLUNMESI
# =============================================================================

# %80 Egitim - %20 Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.20,       # %20 test seti
    random_state=42,      # Tekrarlanabilirlik
    stratify=y            # Sinif dengesini koru
)

print("VERI SETI BOLUNMESI")
print("=" * 50)
print(f"Egitim seti: {X_train.shape[0]:,} ornek ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"Test seti: {X_test.shape[0]:,} ornek ({X_test.shape[0]/len(X)*100:.0f}%)")

print("\nEgitim seti hedef dagilimi:")
print(y_train.value_counts())

print("\nTest seti hedef dagilimi:")
print(y_test.value_counts())

In [ ]:
# =============================================================================
# OZELLIKLERIN OLCEKLENDIRILMESI (STANDARDIZATION)
# =============================================================================

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Ozellikler standardize edildi!")
print(f"Ortalama (egitim): {X_train_scaled.mean():.6f}")
print(f"Std Sapma (egitim): {X_train_scaled.std():.6f}")

## 7.4 Model Egitimi

In [ ]:
# =============================================================================
# MODEL 1: LOJISTIK REGRESYON
# =============================================================================

print("MODEL 1: LOJISTIK REGRESYON")
print("=" * 50)

# Model olustur ve egit
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Tahmin yap
y_pred_log = log_reg.predict(X_test_scaled)
y_prob_log = log_reg.predict_proba(X_test_scaled)[:, 1]

# Dogruluk
acc_log = accuracy_score(y_test, y_pred_log)
print(f"Dogruluk (Accuracy): {acc_log*100:.2f}%")

# Cross-validation
cv_scores_log = cross_val_score(log_reg, X_train_scaled, y_train, cv=5)
print(f"Cross-Validation (5-fold): {cv_scores_log.mean()*100:.2f}% (+/- {cv_scores_log.std()*2*100:.2f}%)")

In [ ]:
# =============================================================================
# MODEL 2: KARAR AGACI
# =============================================================================

print("\nMODEL 2: KARAR AGACI")
print("=" * 50)

# Model olustur ve egit
dt_model = DecisionTreeClassifier(max_depth=10, min_samples_split=20, random_state=42)
dt_model.fit(X_train, y_train)  # Karar agaci icin scaling gerekmez

# Tahmin yap
y_pred_dt = dt_model.predict(X_test)
y_prob_dt = dt_model.predict_proba(X_test)[:, 1]

# Dogruluk
acc_dt = accuracy_score(y_test, y_pred_dt)
print(f"Dogruluk (Accuracy): {acc_dt*100:.2f}%")

# Cross-validation
cv_scores_dt = cross_val_score(dt_model, X_train, y_train, cv=5)
print(f"Cross-Validation (5-fold): {cv_scores_dt.mean()*100:.2f}% (+/- {cv_scores_dt.std()*2*100:.2f}%)")

In [ ]:
# =============================================================================
# MODEL 3: RANDOM FOREST
# =============================================================================

print("\nMODEL 3: RANDOM FOREST")
print("=" * 50)

# Model olustur ve egit
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Tahmin yap
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

# Dogruluk
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f"Dogruluk (Accuracy): {acc_rf*100:.2f}%")

# Cross-validation
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=5)
print(f"Cross-Validation (5-fold): {cv_scores_rf.mean()*100:.2f}% (+/- {cv_scores_rf.std()*2*100:.2f}%)")

---

# BOLUM 8: MODEL DEGERLENDIRME

## 8.1 Model Karsilastirmasi

In [ ]:
# =============================================================================
# MODEL KARSILASTIRMA TABLOSU
# =============================================================================

karsilastirma = pd.DataFrame({
    'Model': ['Lojistik Regresyon', 'Karar Agaci', 'Random Forest'],
    'Dogruluk (%)': [acc_log*100, acc_dt*100, acc_rf*100],
    'CV Ortalama (%)': [cv_scores_log.mean()*100, cv_scores_dt.mean()*100, cv_scores_rf.mean()*100],
    'CV Std (%)': [cv_scores_log.std()*100, cv_scores_dt.std()*100, cv_scores_rf.std()*100]
})

print("MODEL KARSILASTIRMASI")
print("=" * 60)
display(karsilastirma.round(2))

In [ ]:
# =============================================================================
# MODEL KARSILASTIRMA GRAFIGI
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Dogruluk karsilastirmasi
modeller = ['Lojistik\nRegresyon', 'Karar\nAgaci', 'Random\nForest']
dogruluklar = [acc_log*100, acc_dt*100, acc_rf*100]
colors = ['#3498db', '#2ecc71', '#e74c3c']

bars = axes[0].bar(modeller, dogruluklar, color=colors, edgecolor='black')
axes[0].set_ylabel('Dogruluk (%)')
axes[0].set_title('Model Dogruluk Karsilastirmasi', fontweight='bold')
axes[0].set_ylim(0, 100)
axes[0].axhline(y=50, color='gray', linestyle='--', alpha=0.5, label='Rastgele Tahmin (%50)')

for bar, val in zip(bars, dogruluklar):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=12)

# 2. Cross-validation karsilastirmasi
cv_means = [cv_scores_log.mean()*100, cv_scores_dt.mean()*100, cv_scores_rf.mean()*100]
cv_stds = [cv_scores_log.std()*100, cv_scores_dt.std()*100, cv_scores_rf.std()*100]

bars = axes[1].bar(modeller, cv_means, yerr=cv_stds, color=colors, edgecolor='black', capsize=5)
axes[1].set_ylabel('Cross-Validation Dogruluk (%)')
axes[1].set_title('Cross-Validation Karsilastirmasi (5-Fold)', fontweight='bold')
axes[1].set_ylim(0, 100)

for bar, val in zip(bars, cv_means):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3,
                 f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

## 8.2 Confusion Matrix (Karisiklik Matrisi)

In [ ]:
# =============================================================================
# CONFUSION MATRIX GORSELLISTIRMESI
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Model isimleri ve tahminler
modeller_cm = [
    ('Lojistik Regresyon', y_pred_log, acc_log),
    ('Karar Agaci', y_pred_dt, acc_dt),
    ('Random Forest', y_pred_rf, acc_rf)
]

cmap_list = ['Blues', 'Greens', 'Reds']

for idx, (isim, tahmin, dogruluk) in enumerate(modeller_cm):
    cm = confusion_matrix(y_test, tahmin)
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap_list[idx], ax=axes[idx],
                xticklabels=['OnTime', 'Late'], yticklabels=['OnTime', 'Late'],
                annot_kws={'size': 14})
    axes[idx].set_xlabel('Tahmin Edilen', fontsize=11)
    axes[idx].set_ylabel('Gercek', fontsize=11)
    axes[idx].set_title(f'{isim}\nDogruluk: {dogruluk*100:.2f}%', fontweight='bold')

plt.tight_layout()
plt.show()

## 8.3 Siniflandirma Raporu

In [ ]:
# =============================================================================
# DETAYLI SINIFLANDIRMA RAPORU
# =============================================================================

print("LOJISTIK REGRESYON - Siniflandirma Raporu")
print("=" * 55)
print(classification_report(y_test, y_pred_log, target_names=['OnTime', 'Late']))

print("\nKARAR AGACI - Siniflandirma Raporu")
print("=" * 55)
print(classification_report(y_test, y_pred_dt, target_names=['OnTime', 'Late']))

print("\nRANDOM FOREST - Siniflandirma Raporu")
print("=" * 55)
print(classification_report(y_test, y_pred_rf, target_names=['OnTime', 'Late']))

## 8.4 ROC Egrisi ve AUC

In [ ]:
# =============================================================================
# ROC EGRISI
# =============================================================================

plt.figure(figsize=(10, 8))

# Lojistik Regresyon
fpr_log, tpr_log, _ = roc_curve(y_test, y_prob_log)
auc_log = auc(fpr_log, tpr_log)
plt.plot(fpr_log, tpr_log, label=f'Lojistik Regresyon (AUC = {auc_log:.3f})', linewidth=2)

# Karar Agaci
fpr_dt, tpr_dt, _ = roc_curve(y_test, y_prob_dt)
auc_dt = auc(fpr_dt, tpr_dt)
plt.plot(fpr_dt, tpr_dt, label=f'Karar Agaci (AUC = {auc_dt:.3f})', linewidth=2)

# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
auc_rf = auc(fpr_rf, tpr_rf)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.3f})', linewidth=2)

# Rastgele siniflandirici cizgisi
plt.plot([0, 1], [0, 1], 'k--', label='Rastgele Siniflandirici', alpha=0.5)

plt.xlabel('False Positive Rate (Yanlis Pozitif Orani)', fontsize=12)
plt.ylabel('True Positive Rate (Dogru Pozitif Orani)', fontsize=12)
plt.title('ROC Egrisi Karsilastirmasi', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nAUC Skorlari:")
print(f"  Lojistik Regresyon: {auc_log:.4f}")
print(f"  Karar Agaci: {auc_dt:.4f}")
print(f"  Random Forest: {auc_rf:.4f}")

## 8.5 Ozellik Onemliligi (Feature Importance)

In [ ]:
# =============================================================================
# RANDOM FOREST OZELLIK ONEMLILIGI
# =============================================================================

# Ozellik onemliligini alalim
ozellik_onemliligi = pd.DataFrame({
    'Ozellik': X.columns,
    'Onem': rf_model.feature_importances_
}).sort_values('Onem', ascending=True)

# Gorsellestirme
plt.figure(figsize=(10, 8))
colors = plt.cm.RdYlGn(ozellik_onemliligi['Onem'] / ozellik_onemliligi['Onem'].max())
plt.barh(ozellik_onemliligi['Ozellik'], ozellik_onemliligi['Onem'], color=colors, edgecolor='black')
plt.xlabel('Onem Derecesi', fontsize=12)
plt.ylabel('Ozellik', fontsize=12)
plt.title('Random Forest - Ozellik Onemliligi', fontsize=14, fontweight='bold')

# Degerleriyazalim
for i, (idx, row) in enumerate(ozellik_onemliligi.iterrows()):
    plt.text(row['Onem'] + 0.005, i, f'{row["Onem"]*100:.1f}%', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nEN ONEMLI 5 OZELLIK:")
print("=" * 40)
for i, (_, row) in enumerate(ozellik_onemliligi.tail(5).iloc[::-1].iterrows(), 1):
    print(f"{i}. {row['Ozellik']}: %{row['Onem']*100:.2f}")

---

# BOLUM 9: SONUC VE YORUMLAR

## 9.1 Proje Ozeti

In [ ]:
# =============================================================================
# PROJE OZETI
# =============================================================================

print("="*70)
print("                         PROJE OZETI                         ")
print("="*70)

print(f"""
1. VERI SETI BILGILERI
   ----------------------------------------
   - Toplam tablo sayisi: 7
   - Ana tablo (Fulfillment): {len(fulfillment):,} kayit
   - Birlesitirilmis veri seti: {len(df):,} kayit
   - Kullanilan ozellik sayisi: {len(X.columns)}

2. KESIFSEL VERI ANALIZI (EDA)
   ----------------------------------------
   - Teslimat durumu: OnTime ve Late
   - Gecikme orani: %{df['Hedef'].mean()*100:.2f}
   - Kargo firmalari: {df['Carrier'].nunique()} farkli firma
   - Hizmet seviyeleri: {df['ServiceLevel'].nunique()} farkli seviye

3. OZELLIK MUHENDISLIGI
   ----------------------------------------
   - Sure bazli: TasimaSuresi, TaahhutSuresi, GecikmeGunu
   - Tarih bazli: SiparisAyi, SiparisGunu, SiparisSaati, HaftaSonu
   - Finansal: KarMarji, IndirimOrani, KargoGelirOrani

4. MAKINE OGRENMESI MODELLERI
   ----------------------------------------
   | Model               | Dogruluk   | AUC      |
   |---------------------|------------|----------|
   | Lojistik Regresyon  | %{acc_log*100:.2f}     | {auc_log:.4f}   |
   | Karar Agaci         | %{acc_dt*100:.2f}     | {auc_dt:.4f}   |
   | Random Forest       | %{acc_rf*100:.2f}     | {auc_rf:.4f}   |

5. EN IYI MODEL
   ----------------------------------------
""")

# En iyi modeli belirle
en_iyi_idx = np.argmax([acc_log, acc_dt, acc_rf])
en_iyi_isim = ['Lojistik Regresyon', 'Karar Agaci', 'Random Forest'][en_iyi_idx]
en_iyi_acc = [acc_log, acc_dt, acc_rf][en_iyi_idx]
en_iyi_auc = [auc_log, auc_dt, auc_rf][en_iyi_idx]

print(f"   En Basarili Model: {en_iyi_isim}")
print(f"   Dogruluk: %{en_iyi_acc*100:.2f}")
print(f"   AUC: {en_iyi_auc:.4f}")

print("\n" + "="*70)

## 9.2 Teslimat Gecikmesini Etkileyen Faktorler

In [ ]:
# =============================================================================
# GECIKMEYI ETKILEYEN FAKTORLER
# =============================================================================

print("TESLIMAT GECIKMESINI ETKILEYEN TEMEL FAKTORLER")
print("="*55)

# En onemli 5 faktor
en_onemliler = ozellik_onemliligi.tail(5).iloc[::-1]

for i, (_, row) in enumerate(en_onemliler.iterrows(), 1):
    yorum = ""
    if 'Carrier' in row['Ozellik']:
        yorum = "Kargo firmasi secimi teslimat performansini dogrudan etkiler"
    elif 'ServiceLevel' in row['Ozellik']:
        yorum = "Hizmet seviyesi (express, standard) teslimat hizini belirler"
    elif 'ShipCost' in row['Ozellik']:
        yorum = "Kargo maliyeti hizmet kalitesi ile iliskilidir"
    elif 'ToplamGelir' in row['Ozellik'] or 'Gelir' in row['Ozellik']:
        yorum = "Buyuk siparisler ozel ilgi gerektirebilir"
    elif 'Saat' in row['Ozellik']:
        yorum = "Siparis saati lojistik planlamayi etkiler"
    elif 'Ay' in row['Ozellik']:
        yorum = "Mevsimsellik ve yogun donemler etkilidir"
    elif 'Gun' in row['Ozellik']:
        yorum = "Hafta ici/sonu siparis zamani onemlidir"
    else:
        yorum = "Model tarafindan onemli bulunan faktor"
    
    print(f"\n{i}. {row['Ozellik']} (Onem: %{row['Onem']*100:.1f})")
    print(f"   -> {yorum}")

## 9.3 Is Onerileri

In [ ]:
# =============================================================================
# IS ONERILERI
# =============================================================================

print("IS ONERILERI")
print("="*55)

oneriler = [
    "1. KARGO FIRMASI OPTIMIZASYONU",
    "   - Dusuk gecikme oranina sahip kargo firmalari tercih edilmeli",
    "   - Performans bazli kargo firmasi secimi yapilmali",
    "",
    "2. HIZMET SEVIYESI YONETIMI",
    "   - Express teslimat yogun donemlerde onceliklendirilmeli",
    "   - Musteri beklentilerine uygun hizmet seviyesi sunulmali",
    "",
    "3. MEVSIMSEL PLANLAMA",
    "   - Black Friday ve Noel oncesi kapasite artirilmali",
    "   - Yogun donemlerde ek lojistik kaynaklar hazirlanmali",
    "",
    "4. TAHMINE DAYALI LOJISTIK",
    "   - ML modeli ile yuksek gecikme riski olan siparisler tespit edilmeli",
    "   - Proaktif mudahale ile musteriler bilgilendirilmeli",
    "",
    "5. MUSTERI ILETISIMI",
    "   - Gecikme riski yuksek siparislerde erken bildirim yapilmali",
    "   - Alternatif teslimat secenekleri sunulmali"
]

for oneri in oneriler:
    print(oneri)

## 9.4 Veri Setinin Guclu ve Sinirli Yonleri

### Guclu Yonler:
- Coklu tablo yapisi sayesinde kapsamli bir e-ticaret senaryosu sunmasi
- Black Friday, Cyber Monday ve Noel gibi yogun donemleri icermesi
- Teslimat, iade ve kampanya bilgilerinin bir arada bulunmasi
- Gercekci lojistik verileri icermesi
- 45,000+ siparis kaydinin bulunmasi

### Sinirli Yonler:
- Cografi bilgilerin (mesafe, bolge) sinirli olmasi
- Hava durumu gibi dis faktorlerin bulunmamasi
- Musteri davranis verilerinin eksikligi
- Trafik ve ulasim kosullarinin dahil edilmemesi

---

## Kaynaklar

- **Veri Seti:** Kaggle - Christmas Retail Sales - Shipping & Delivery Dataset
- **Python Kutuphaneleri:** pandas, numpy, matplotlib, seaborn, scikit-learn, scipy

---

*Bu notebook, Veri Bilimine Giris ve Makine Ogrenmesi dersi kapsaminda hazirlanmistir.*